## Assignment 2.1

In [2]:
import math
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import string
import numpy as np
import pandas as pd
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [2]:
corpus = [
    'the sun is a star',
    'the moon is a satellite',
    'the sun and moon are celestial bodies'
]

In [3]:
tokenized_corpus = [sentence.lower().split() for sentence in corpus]

vocab = sorted(set(word for sentence in tokenized_corpus for word in sentence))
print(f"Vocabulary: {vocab}")

Vocabulary: ['a', 'and', 'are', 'bodies', 'celestial', 'is', 'moon', 'satellite', 'star', 'sun', 'the']


In [12]:
tf = []
for sentence in tokenized_corpus:
    word_counts = {}
    total_words = len(sentence)
    for word in vocab:
        word_counts[word] = sentence.count(word) / total_words
    tf.append(word_counts)

print(tf)

[{'a': 0.2, 'and': 0.0, 'are': 0.0, 'bodies': 0.0, 'celestial': 0.0, 'is': 0.2, 'moon': 0.0, 'satellite': 0.0, 'star': 0.2, 'sun': 0.2, 'the': 0.2}, {'a': 0.2, 'and': 0.0, 'are': 0.0, 'bodies': 0.0, 'celestial': 0.0, 'is': 0.2, 'moon': 0.2, 'satellite': 0.2, 'star': 0.0, 'sun': 0.0, 'the': 0.2}, {'a': 0.0, 'and': 0.14285714285714285, 'are': 0.14285714285714285, 'bodies': 0.14285714285714285, 'celestial': 0.14285714285714285, 'is': 0.0, 'moon': 0.14285714285714285, 'satellite': 0.0, 'star': 0.0, 'sun': 0.14285714285714285, 'the': 0.14285714285714285}]


In [ ]:
N = len(tokenized_corpus)
idf = {}

for word in vocab:
    df = sum(1 for sentence in tokenized_corpus if word in sentence)
    idf[word] = math.log((N) / (1 + df)) + 1

tfidf_manual = []
for doc_tf in tf:
    doc_tfidf = {}
    for word in vocab:
        doc_tfidf[word] = doc_tf[word] * idf[word]
    tfidf_manual.append(doc_tfidf)


print("\nManual TF-IDF Results:")
for i, doc_tfidf in enumerate(tfidf_manual):
    print(f"\nDocument {i+1}:")
    print(doc_tfidf)



Manual TF-IDF Results:

Document 1:
{'a': 0.2, 'and': 0.0, 'are': 0.0, 'bodies': 0.0, 'celestial': 0.0, 'is': 0.2, 'moon': 0.0, 'satellite': 0.0, 'star': 0.2810930216216329, 'sun': 0.2, 'the': 0.14246358550964383}

Document 2:
{'a': 0.2, 'and': 0.0, 'are': 0.0, 'bodies': 0.0, 'celestial': 0.0, 'is': 0.2, 'moon': 0.2, 'satellite': 0.2810930216216329, 'star': 0.0, 'sun': 0.0, 'the': 0.14246358550964383}

Document 3:
{'a': 0.0, 'and': 0.20078072972973776, 'are': 0.20078072972973776, 'bodies': 0.20078072972973776, 'celestial': 0.20078072972973776, 'is': 0.0, 'moon': 0.14285714285714285, 'satellite': 0.0, 'star': 0.0, 'sun': 0.14285714285714285, 'the': 0.10175970393545987}


In [10]:
cv = CountVectorizer()
cv_matrix = cv.fit_transform(corpus)
print("\nCountVectorizer Vocabulary:", cv.get_feature_names_out())
print("CountVectorizer Matrix:\n", cv_matrix.toarray())

tv = TfidfVectorizer()
tv_matrix = tv.fit_transform(corpus)
print("\nTfidfVectorizer Vocabulary:", tv.get_feature_names_out())
print("TfidfVectorizer Matrix:\n", tv_matrix.toarray())



CountVectorizer Vocabulary: ['and' 'are' 'bodies' 'celestial' 'is' 'moon' 'satellite' 'star' 'sun'
 'the']
CountVectorizer Matrix:
 [[0 0 0 0 1 0 0 1 1 1]
 [0 0 0 0 1 1 1 0 0 1]
 [1 1 1 1 0 1 0 0 1 1]]

TfidfVectorizer Vocabulary: ['and' 'are' 'bodies' 'celestial' 'is' 'moon' 'satellite' 'star' 'sun'
 'the']
TfidfVectorizer Matrix:
 [[0.         0.         0.         0.         0.4804584  0.
  0.         0.63174505 0.4804584  0.37311881]
 [0.         0.         0.         0.         0.4804584  0.4804584
  0.63174505 0.         0.         0.37311881]
 [0.4261835  0.4261835  0.4261835  0.4261835  0.         0.32412354
  0.         0.         0.32412354 0.25171084]]


## Assignment 2.2

### Problem 1

In [7]:
df = pd.read_csv('spam.csv', encoding='latin-1')[['v1', 'v2']]
df.columns = ['Label', 'Message']

In [51]:
df

,Label,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [62]:
df.iloc[0][1]

C:\Users\aniru\AppData\Local\Temp\ipykernel_21436\2309508539.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.iloc[0][1]


'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [8]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aniru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aniru\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# Load pre-trained Word2Vec model
print("Loading Word2Vec model... (this may take a few minutes)")
w2v_model = api.load("word2vec-google-news-300")
print("Word2Vec loaded.")

Loading Word2Vec model... (this may take a few minutes)
Word2Vec loaded.


In [10]:
df['Label'] = df['Label'].map({'ham': 0, 'spam': 1})

In [ ]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

def text_to_vector(tokens, w2v, vector_size=300):
    vectors = [w2v[word] for word in tokens if word in w2v]
    if vectors:return np.mean(vectors, axis=0)
        
    else:
        return np.zeros(vector_size)

In [12]:
X_vectors = np.vstack([text_to_vector(preprocess_text(msg), w2v_model) 
                       for msg in df['Message']])
y = df['Label']

In [75]:
tokens = preprocess_text(df['Message'][0])
vectors = [w2v_model[word] for word in tokens if word in w2v_model]
vectors
np.mean(vectors, axis=0)

array([-0.01980591,  0.05167062,  0.02709961,  0.21868025, -0.0310342 ,
        0.03897967,  0.08196586, -0.09603446,  0.01432146,  0.07736329,
       -0.06193366, -0.1637486 , -0.02575248, -0.0471889 , -0.10367257,
        0.14072964,  0.15539551,  0.09352984,  0.03390067, -0.02132089,
       -0.0716391 , -0.01377705,  0.09657506,  0.03925433, -0.01184082,
        0.0204869 , -0.12768555,  0.02552141,  0.0472363 , -0.04317801,
       -0.05688912,  0.02345221, -0.05597796, -0.01293073, -0.01777867,
       -0.04088129, -0.02099078, -0.02489035,  0.02235821,  0.03364781,
        0.03664725, -0.09109061,  0.13033621,  0.05961827,  0.02466256,
       -0.08101545, -0.09543718, -0.05844443,  0.01415144,  0.10117885,
       -0.0839059 ,  0.15156338, -0.02831377,  0.01769148, -0.05941882,
        0.08585031, -0.09414673, -0.1260376 ,  0.04568917, -0.09597342,
       -0.09876796,  0.047869  , -0.09463065, -0.06553432,  0.01525879,
       -0.10970198, -0.07511684,  0.04650007, -0.01913779,  0.10

In [46]:
X_vectors[0]

array([-0.01980591,  0.05167062,  0.02709961,  0.21868025, -0.0310342 ,
        0.03897967,  0.08196586, -0.09603446,  0.01432146,  0.07736329,
       -0.06193366, -0.16374861, -0.02575248, -0.0471889 , -0.10367257,
        0.14072964,  0.15539551,  0.09352984,  0.03390067, -0.02132089,
       -0.0716391 , -0.01377705,  0.09657506,  0.03925433, -0.01184082,
        0.0204869 , -0.12768555,  0.02552141,  0.0472363 , -0.04317801,
       -0.05688912,  0.02345221, -0.05597796, -0.01293073, -0.01777867,
       -0.04088129, -0.02099078, -0.02489035,  0.02235821,  0.03364781,
        0.03664725, -0.09109061,  0.13033621,  0.05961827,  0.02466256,
       -0.08101545, -0.09543718, -0.05844443,  0.01415144,  0.10117885,
       -0.0839059 ,  0.15156338, -0.02831377,  0.01769148, -0.05941882,
        0.08585031, -0.09414673, -0.1260376 ,  0.04568917, -0.09597342,
       -0.09876796,  0.047869  , -0.09463065, -0.06553432,  0.01525879,
       -0.10970198, -0.07511684,  0.04650007, -0.01913779,  0.10

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X_vectors, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.2f}")

Test Accuracy: 0.94


In [26]:
def predict_message_class(model, w2v, message):
    tokens = preprocess_text(message)
    vec = text_to_vector(tokens, w2v).reshape(1, -1)
    pred = model.predict(vec)[0]
    return 'spam' if pred == 1 else 'ham'


In [27]:
print(predict_message_class(clf, w2v_model, "click and claim reward now"))

spam


### Problem 2

In [15]:
df2 = pd.read_csv('Tweets.csv')[['airline_sentiment', 'text']]

In [ ]:
lemmatizer = WordNetLemmatizer()

def preprocess_tweet(text):    
    text = text.lower()
    contractions = {"don't": "do not", "can't": "cannot", "won't": "will not"}
    for k, v in contractions.items():
        text = text.replace(k, v)
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # URLs
    text = re.sub(r"@\w+", "", text)  # Mentions
    text = re.sub(r"#\w+", "", text)  # Hashtags
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

In [31]:
X_vectors_2 = np.vstack([
    text_to_vector(preprocess_tweet(tweet), w2v_model)
    for tweet in df2['text']
])
y2 = df2['airline_sentiment']

In [32]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X_vectors_2, y2, test_size=0.2, random_state=42, stratify=y2)

clf2 = LogisticRegression(max_iter=1000, multi_class='multinomial')
clf2.fit(X_train2, y_train2)
y_pred2 = clf2.predict(X_test2)

print(f"Test Accuracy: {accuracy_score(y_test2, y_pred2):.2f}")


Test Accuracy: 0.76


In [ ]:
def predict_tweet_sentiment(model, w2v, tweet):
    tokens = preprocess_tweet(tweet)
    vec = text_to_vector(tokens, w2v).reshape(1, -1)
    return model.predict(vec)


In [39]:
print(predict_tweet_sentiment(clf2, w2v_model, "hated it"))


['negative']
